# Population

> Data Structures for Population

In [1]:
#| default_exp population

In [2]:
#| hide
from nbdev.showdoc import *


In [3]:
#| echo: false
from nbdev.showdoc import show_doc

In [4]:
#| export
from dataclasses import field
from typing import List, Optional, Dict, Callable, Any

import jax
import jax.numpy as jnp

from chewc.sp import SimParam
from typing import Tuple
from numpy.random import default_rng
import msprime
import tskit
import numpy as np
import random
from collections import defaultdict
from flax.struct import dataclass as flax_dataclass, field

#testing
import jax
import jax.numpy as jnp
from fastcore.test import test_eq, test_ne

In [5]:
#| export
@flax_dataclass(frozen=True) # Make the class immutable, a JAX best practice
class Population:
    """
    A container for all data related to a population of individuals, designed
    for JAX-based genetic simulations.

    This structure is immutable. All operations that modify a population should
    return a new Population object.

    Attributes:
        geno (jnp.ndarray): A 4D array representing the genotypes of the population.
            Shape: `(nInd, nChr, ploidy, nLoci)`. dtype: `jnp.uint8`.
        idb (jnp.ndarray): A 4D array representing the founder origins of each allele of the population.
            Shape: `(nInd, nChr, ploidy, nLoci)`. dtype: `jnp.uint8`.        id (jnp.ndarray): The primary, user-facing identifier for each individual.
            These IDs may not be contiguous or sorted. Shape: `(nInd,)`.
        iid (jnp.ndarray): The internal, zero-indexed, contiguous identifier.
            Crucial for robust indexing in JAX operations. Shape: `(nInd,)`.
        mother (jnp.ndarray): Array of internal IDs (`iid`) for the mother of each
            individual. A value of -1 indicates no known mother. Shape: `(nInd,)`.
        father (jnp.ndarray): Array of internal IDs (`iid`) for the father of each
            individual. A value of -1 indicates no known father. Shape: `(nInd,)`.
        sex (jnp.ndarray): The sex of each individual, represented numerically
            (e.g., 0 for male, 1 for female). dtype: `jnp.int8`. Shape: `(nInd,)`.
        gen (jnp.ndarray): The generation each individual is in, represented numerically
             dtype: `jnp.int8`. Shape: `(nInd,)`.

        pheno (jnp.ndarray): The phenotypic values for each individual.
            Shape: `(nInd, nTraits)`.
        fixEff (jnp.ndarray): The value of a fixed effect for each individual,
            often used as an intercept in genomic selection models. Shape: `(nInd,)`.
        
        bv (Optional[jnp.ndarray]): The true breeding values (additive genetic effects)
            for each individual. Shape: `(nInd, nTraits)`.
        dd (Optional[jnp.ndarray]): The true dominance deviations for each individual.
            Shape: `(nInd, nTraits)`.
        aa (Optional[jnp.ndarray]): The true additive-by-additive epistatic deviations
            for each individual. Shape: `(nInd, nTraits)`.

        ebv (Optional[jnp.ndarray]): The estimated breeding values for each
            individual. Shape: `(nInd, nTraits)`.
        gxe (Optional[jnp.ndarray]): Genotype-by-environment interaction effects.
            Shape depends on the specific GxE model.

        misc (Dict): A dictionary for storing miscellaneous, non-JAX-critical
            metadata about individuals. Static.
        miscPop (Dict): A dictionary for storing miscellaneous, non-JAX-critical
            metadata about the entire population. Static.
    """
    # --- Core Genotype Info ---
    geno: jnp.ndarray
    ibd : jnp.ndarray
    
    # --- Pedigree and Identifiers ---
    id: jnp.ndarray
    iid: jnp.ndarray
    mother: jnp.ndarray
    father: jnp.ndarray
    sex: jnp.ndarray
    gen: jnp.ndarray

    # --- Trait and Value Data ---
    pheno: jnp.ndarray
    fixEff: jnp.ndarray
    
    # --- JAX Compatibility ---
    # Boolean mask to distinguish real vs. padded individuals
    is_active: jnp.ndarray

    gv: Optional[jnp.ndarray] = None      # Genetic Value (BV + Intercept)
    bv: Optional[jnp.ndarray] = None      # Breeding Value (Additive)
    dd: Optional[jnp.ndarray] = None      # Dominance Deviations
    aa: Optional[jnp.ndarray] = None      # Additive-by-Additive Epistatic Deviations
    
    ebv: Optional[jnp.ndarray] = None
    gxe: Optional[jnp.ndarray] = None

    # --- Metadata ---
    misc: Optional[Dict[str, Any]] = field(default=None, pytree_node=False)
    miscPop: Optional[Dict[str, Any]] = field(default=None, pytree_node=False)




    # def __post_init__(self):
    #     """Validates the consistency of the population data after initialization."""
    #     # --- START: THE ONLY CHANGE NEEDED IN THIS FILE ---
    #     # During a JAX trace (jit, vmap, scan), `self.geno` is a Tracer
    #     # object without a concrete shape. This guard bypasses validation
    #     # during tracing by checking for an attribute that only tracers have.
    #     if isinstance(self.geno, Tracer):
    #         return

    #     # --- END: THE ONLY CHANGE NEEDED IN THIS FILE ---

    #     # The rest of your validation code will now only run in eager mode.
    #     n_ind = self.nInd
    #     # Validate pedigree and identifier shapes
    #     assert self.iid.shape[0] == n_ind, f"Internal ID array length ({self.iid.shape[0]}) must match number of individuals ({n_ind})."
    #     assert self.mother.shape[0] == n_ind, f"Mother array length ({self.mother.shape[0]}) must match number of individuals ({n_ind})."
    #     assert self.father.shape[0] == n_ind, f"Father array length ({self.father.shape[0]}) must match number of individuals ({n_ind})."
    #     assert self.sex.shape[0] == n_ind, f"Sex array length ({self.sex.shape[0]}) must match number of individuals ({n_ind})."
        
    #     # Validate trait-related array shapes
    #     assert self.pheno.shape[0] == n_ind, f"Phenotype (pheno) array length ({self.pheno.shape[0]}) must match number of individuals ({n_ind})."
    #     assert self.fixEff.shape[0] == n_ind, f"Fixed effect (fixEff) array length ({self.fixEff.shape[0]}) must match number of individuals ({n_ind})."
        
    #     # Validate optional genetic value components
    #     if self.bv is not None:
    #         assert self.bv.shape[0] == n_ind, f"Breeding value (bv) array length ({self.bv.shape[0]}) must match number of individuals ({n_ind})."
    #     if self.dd is not None:
    #         assert self.dd.shape[0] == n_ind, f"Dominance deviation (dd) array length ({self.dd.shape[0]}) must match number of individuals ({n_ind})."
    #     if self.aa is not None:
    #         assert self.aa.shape[0] == n_ind, f"Epistatic deviation (aa) array length ({self.aa.shape[0]}) must match number of individuals ({n_ind})."
    #     if self.ebv is not None:
    #         assert self.ebv.shape[0] == n_ind, f"EBV array length ({self.ebv.shape[0]}) must match number of individuals ({n_ind})."

    @property
    def nInd(self) -> int:
        """Returns the number of ACTIVE individuals in the population."""
        return jnp.sum(self.is_active)

    @property
    def max_pop_size(self) -> int:
        """Returns the maximum capacity of the population arrays."""
        return self.geno.shape[0]

    @property
    def nTraits(self) -> int:
        """Returns the number of traits, inferred from the breeding value (bv) shape."""
        if self.bv is None or self.bv.ndim <= 1:
            return 0
        return self.bv.shape[1]
    
    @property
    def haplo_matrix(self) -> jnp.ndarray:
        """ returns a haplotype matrix of shape (n_ind*ploidy, n_chr*n_markers)"""
        return self.geno.transpose(0, 2, 1, 3).reshape(self.geno.shape[0] * self.geno.shape[2], -1)

    @property
    def dosage(self) -> jnp.ndarray:
        """
        Calculates the dosage of alternate alleles for each individual.
        """
        dosage_per_chr = jnp.sum(self.geno, axis=2)
        return dosage_per_chr.reshape(self.nInd, -1)
    
    def plot_maf(self, genetic_map=None, maf_threshold=None):
        """
        Plot MAF distribution as a quick sanity check for the population.
        """
        import matplotlib.pyplot as plt
        
        maf_values = []
        
        # Calculate MAF for each marker
        for chr_idx in range(self.nChr):
            for snp_idx in range(self.geno.shape[3]):
                if genetic_map is not None and jnp.isnan(genetic_map[chr_idx, snp_idx]):
                    continue
                
                marker_genotypes = self.geno[:, chr_idx, :, snp_idx]
                if jnp.any(jnp.isnan(marker_genotypes)):
                    continue
                
                allele_freq = float(jnp.mean(marker_genotypes))
                maf = min(allele_freq, 1 - allele_freq)
                maf_values.append(maf)
        
        if not maf_values:
            print("No valid markers found!")
            return
        
        plt.figure(figsize=(8, 5))
        plt.hist(maf_values, bins=50, alpha=0.7, edgecolor='black')
        plt.xlabel('Minor Allele Frequency (MAF)')
        plt.ylabel('Number of Markers')
        plt.title('MAF Distribution')
        
        mean_maf = jnp.mean(jnp.array(maf_values))
        plt.axvline(mean_maf, color='red', linestyle='--', label=f'Mean: {mean_maf:.3f}')
        
        if maf_threshold is not None:
            plt.axvline(maf_threshold, color='green', linestyle=':', 
                        label=f'Threshold: {maf_threshold}')
        
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        fixed = sum(1 for maf in maf_values if maf == 0)
        print(f"Markers: {len(maf_values)} | Fixed: {fixed} | Mean MAF: {mean_maf:.3f}")

    def __repr__(self) -> str:
        """Provides a concise representation of the Population object."""
        return (f"Population(nInd={self.nInd}, nTraits={self.nTraits}, "
                f"has_ebv={'Yes' if self.ebv is not None else 'No'})")


In [6]:
#| export

# chewc/population.py

def combine_populations(pop1: Population, pop2: Population, max_pop_size: int) -> Population:
    """
    Combines two populations into a single, new padded population.
    """
    n1 = pop1.nInd
    n2 = pop2.nInd
    n_total = n1 + n2

    if n_total > max_pop_size:
        raise ValueError(f"Combined population size ({n_total}) exceeds max_pop_size ({max_pop_size}).")

    # The new iids for pop2 start right after pop1's iids.
    # Note: Parents in pop2's pedigree are still using old pop2 iids (0 to n2-1)
    # We need to offset them by n1 to match their new position in the combined arrays.
    pop2_mother_offset = jnp.where(pop2.mother != -1, pop2.mother + n1, -1)
    pop2_father_offset = jnp.where(pop2.father != -1, pop2.father + n1, -1)

    # --- FIX: Padding Logic ---
    n_pad = max_pop_size - n_total

    def pad(arr, val):
        # Helper to pad the first axis of an array
        pad_width = ((0, n_pad),) + ((0, 0),) * (arr.ndim - 1)
        return jnp.pad(arr, pad_width, 'constant', constant_values=val)

    # Combine active individuals' data
    combined_geno = jnp.concatenate([pop1.geno[:n1], pop2.geno[:n2]])
    combined_ibd = jnp.concatenate([pop1.ibd[:n1], pop2.ibd[:n2]])
    combined_id = jnp.concatenate([pop1.id[:n1], pop2.id[:n2]])
    combined_mother = jnp.concatenate([pop1.mother[:n1], pop2_mother_offset[:n2]])
    combined_father = jnp.concatenate([pop1.father[:n1], pop2_father_offset[:n2]])
    combined_sex = jnp.concatenate([pop1.sex[:n1], pop2.sex[:n2]])
    combined_gen = jnp.concatenate([pop1.gen[:n1], pop2.gen[:n2]])
    combined_pheno = jnp.concatenate([pop1.pheno[:n1], pop2.pheno[:n2]])
    combined_fixEff = jnp.concatenate([pop1.fixEff[:n1], pop2.fixEff[:n2]])
    combined_bv = jnp.concatenate([pop1.bv[:n1], pop2.bv[:n2]]) if pop1.bv is not None and pop2.bv is not None else None
    
    # --- NEW: Combine is_active masks ---
    combined_is_active = jnp.concatenate([pop1.is_active[:n1], pop2.is_active[:n2]])

    # Create the new Population object with padded arrays
    return Population(
        geno=pad(combined_geno, 0),
        ibd=pad(combined_ibd, -1),
        id=pad(combined_id, -1),
        iid=jnp.arange(max_pop_size), # Always 0 to max_pop_size-1
        mother=pad(combined_mother, -1),
        father=pad(combined_father, -1),
        sex=pad(combined_sex, -1),
        gen=pad(combined_gen, -1),
        is_active=pad(combined_is_active, False), # Pad with False
        pheno=pad(combined_pheno, jnp.nan),
        fixEff=pad(combined_fixEff, 0.),
        bv=pad(combined_bv, jnp.nan) if combined_bv is not None else jnp.full((max_pop_size, 0), jnp.nan)
        # Ensure optional fields are handled correctly
    )

def subset_population(pop: Population, indices: jnp.ndarray) -> Population:
    """
    Creates a new Population object containing only the individuals specified by indices.

    Args:
        pop: The original Population object.
        indices: A JAX array of integer indices of individuals to select.

    Returns:
        A new Population object with the subset of individuals.
    """
    n_new_ind = indices.shape[0]

    # Use .at[indices].get() or direct indexing to select rows from each array
    # Ensure all arrays are handled, including optional ones
    return Population(
        geno=pop.geno[indices],
        ibd=pop.ibd[indices],  # ADDED: Handle IBD array
        id=pop.id[indices],
        iid=jnp.arange(n_new_ind, dtype=jnp.int32), # New internal IDs are 0-indexed
        mother=pop.mother[indices],
        father=pop.father[indices],
        sex=pop.sex[indices],
        gen=pop.gen[indices],
        pheno=pop.pheno[indices],
        fixEff=pop.fixEff[indices],
        bv=pop.bv[indices] if pop.bv is not None else None,
        dd=pop.dd[indices] if pop.dd is not None else None,
        aa=pop.aa[indices] if pop.aa is not None else None,
        ebv=pop.ebv[indices] if pop.ebv is not None else None,
        gxe=pop.gxe[indices] if pop.gxe is not None else None,
        # misc and miscPop are static and apply to the whole population,
        # so they are carried over as is.
        misc=pop.misc,
        miscPop=pop.miscPop
    )


In [7]:
#| hide
import nbdev; nbdev.nbdev_export()